In [3]:
import cv2
import numpy as np
import os
import csv
import pickle

print("Libraries imported")

Libraries imported


In [8]:
class preprocesser:

    def __init__(self, input_csv, output_path, labels_path, limit):
        with open(input_csv) as fp:
            rows = list(csv.reader(fp))

        self.X=[]
        self.y=[]

        count = 0
        for row in rows:
            img=cv2.imread(row[0],1)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.X.append(gray)
            if row[0][48]=='p':
                self.y.append(1)
            else:
                self.y.append(0)
            count=count+1
            if count==limit:
                break
        
        self.Xp=self.preprocess(self.X)
        self.savedata(self.Xp, self.y, output_path, labels_path)


    def preprocess(self, X):
        Xp=[]
        for i in range(len(X)):
            img = X[i]
            thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            x,y,w,h=cv2.boundingRect(thresh)
            x,y,w,h=x,y,w+20,h+20
            img = img[y:y+h, x:x+w]
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            Xp.append(clahe.apply(img))
        return Xp
    
    def savedata(self, data, labels, path, labels_path):
        for i in range(len(data)):
            if self.y[i] == 1:
                cv2.imwrite(path+'/abnormal/'+str(i)+".png",data[i])
            else:
                cv2.imwrite(path+'/normal/'+str(i)+".png",data[i])
        with open(labels_path, "wb") as fp:
            pickle.dump(labels, fp)

In [9]:
training_data = "train_image_paths.csv"
training_path = "Dataset/train/"
train_labels = "train_labels.txt"
limit=8379

preprocesser(training_data, training_path, train_labels, limit)

In [10]:
testing_data = "valid_image_paths.csv"
testing_path = "Dataset/test/"
test_labels = "test_labels.txt"
limit = 563

preprocesser(testing_data, testing_path, test_labels, limit)